In [1]:
import escher
import numpy as np
import pandas as pd
import os

## Loop through fluxes directory and generate escher maps

In [2]:
os.makedirs("../results/maps", exist_ok=True)
for filename in os.listdir("../results/dataframe/fluxes/for_escher"):
    if not filename.startswith('.'):
        f = os.path.join("../results/dataframes/fluxes/for_escher", filename)
        if os.path.isdir(f):
            continue
            
        fluxes = pd.read_csv(f, index_col=0, header=[0,1])
        comparison = filename.split(".csv",1)[0]
        
        
        builder = escher.Builder(
            model_json = f"../results/iMT1026-NZ.json",
            map_json = "../data/maps/PichiaCCMap.json")



        builder.reaction_compare_style = 'log2_fold'
        builder.reaction_styles = ['size', 'color']
        builder.enable_tooltips = ['label']
        builder.reaction_no_data_color = ['#26c55a']
        builder.reaction_no_data_size = ['10']

        for index, row in fluxes.iterrows():
            if (min(row) <= 0)&(max(row)==0):
                new_row = row
                new_row[np.argmax(row)] = -1e-7
                fluxes.loc[index] = new_row

        fluxes.replace(to_replace=0, value=0.0000001, inplace=True)
        # log2_array = np.log2(fluxes.iloc[:,0]/fluxes.iloc[:,1]).to_numpy()
        # log2_array = log2_array[~np.isnan(log2_array)]
        # ll, cll, cl, cul, ul = np.percentile(log2_array,[5,25,50,75,95])

        builder.reaction_scale=[
        { 'type': 'value', 'value': -2,  'color': '#000bff', 'size':20},
        { 'type': 'value', 'value': -1, 'color': '#118fd4', 'size':15},
        { 'type': 'value', 'value': -0.5, 'color': '#11ced4', 'size':15},
        { 'type': 'value', 'value': 0,  'color': '#8b8b8b', 'size':10},
        { 'type': 'value', 'value': 0.5, 'color': '#ffd900', 'size':15},
        { 'type': 'value', 'value': 1, 'color': '#ff7700', 'size':15},
        { 'type': 'value', 'value': 2,  'color': '#ff0000', 'size':20},
        ]

        builder.reaction_data = fluxes
        builder.save_html(f"../results/maps/{comparison}Map.html")    
